In [63]:
# Import Dependencies
import pandas as pd
import os

In [71]:
# Import CSV and convert to dataframes
contact_df = pd.read_csv(os.path.join("Resources", "school_contact.csv"))
ratio_df = pd.read_csv(os.path.join("Resources", "school_ratio.csv"))
exp_df = pd.read_csv(os.path.join("Resources", "school_exp.csv"))
test_df = pd.read_csv(os.path.join("Resources", "school_test.csv"))

#### Clean contact dataframe

In [65]:
# Drop unnecessary columns
contact_df = contact_df[["DISTRICT_CODE", "DISTRICT_NAME", "SCHOOL_CODE", "SCHOOL_NAME", "GRADESPAN"]]

# Verify no missing data
contact_df.isnull().sum()

# Verify data are of correct type
contact_df.dtypes

# Add leading zeros to district and school codes
contact_df["DISTRICT_CODE"] = contact_df["DISTRICT_CODE"].apply(lambda x: "{0:0>4}".format(x))
contact_df["SCHOOL_CODE"] = contact_df["SCHOOL_CODE"].apply(lambda x: "{0:0>3}".format(x))

# Create unique key column from district and school codes
contact_df["DS_CODE"] = contact_df["DISTRICT_CODE"].map(str) + "-" + contact_df["SCHOOL_CODE"].map(str)

# Verify no duplicate district-school codes
contact_df[contact_df.duplicated(['DS_CODE'])]

# Verify no incorrect data in grade span
contact_df.GRADESPAN.unique()

array(['05-08', 'PK-04', '09-12', 'PK-08', 'KG-08', 'PK-PK', 'PK-05',
       '06-08', 'KG-02', 'PK-02', '03-05', 'PK-03', '04-08', '04-05',
       'KG-06', '07-08', 'PK-06', 'PK-01', 'KG-05', 'PK-12', 'KG-12',
       '07-12', '01-06', '01-08', '02-04', '03-06', '04-06', 'PK-KG',
       '01-03', '03-08', 'KG-03', '06-12', 'KG-04', '03-03', '01-04',
       '01-12', '02-06', 'KG-01', '06-07', '02-03', '05-06', '06-06',
       '01-02', 'KG-10', 'PK-09', '03-04', '09-10', '05-05', '08-12',
       '05-12', 'PK-07', '08-09', '02-12', '02-05', '01-05', '09-09',
       '10-12', '11-12', '07-09', 'KG-KG', '02-08', '06-09', '03-07',
       '02-02', '12-12', 'PK-10', 'KG-07', 'KG-11', 'KG-09'], dtype=object)

#### Clean ratio dataframe

In [66]:
# Drop and rename columns
ratio_df = ratio_df[["DistrictCode", "SchoolCode", "Student_Teacher_School", "Student_Teacher_District"]]
ratio_df = ratio_df.rename(columns={"DistrictCode": "DISTRICT_CODE", "SchoolCode": "SCHOOL_CODE", \
                                    "Student_Teacher_School": "SCH_RAT", "Student_Teacher_District": "DIST_RAT"})

# Verify no missing data
ratio_df.isnull().sum()

# Verify data are of correct type
ratio_df.dtypes

# Add leading zeros to district and school codes
ratio_df["DISTRICT_CODE"] = ratio_df["DISTRICT_CODE"].apply(lambda x: "{0:0>4}".format(x))
ratio_df["SCHOOL_CODE"] = ratio_df["SCHOOL_CODE"].apply(lambda x: "{0:0>3}".format(x))

# Create unique key column from district and school codes
ratio_df["DS_CODE"] = ratio_df["DISTRICT_CODE"].map(str) + "-" + ratio_df["SCHOOL_CODE"].map(str)

# Verify no duplicate district-school codes
ratio_df[ratio_df.duplicated(['DS_CODE'])]

# Review ratios to verify no junk data
ratio_df.SCH_RAT.unique()

# Replaced "N" values and likely inaccurate values with None
ratio_df = ratio_df.replace(["148:1", "53:1", "56:1", "50:1", "152:1", "66:1", "106:1", "74:1",\
                             "245:1", "268:1", "520:1", "327:1", "N"], None)

# Convert ratios to a single number and cast them as integers
temp_df = ratio_df["SCH_RAT"].str.split(":", n = 1, expand = True)
ratio_df["SCH_RAT"] = temp_df[0].astype('int64')

temp_df = ratio_df["DIST_RAT"].str.split(":", n = 1, expand = True)
ratio_df["DIST_RAT"] = temp_df[0].astype('int64')

# Verify values are of the correct type
ratio_df.dtypes

DISTRICT_CODE    object
SCHOOL_CODE      object
SCH_RAT           int64
DIST_RAT          int64
DS_CODE          object
dtype: object

#### Clean expense dataframe

In [67]:
# Drop and rename columns
exp_df = exp_df[["DistrictCode", "Total"]]
exp_df = exp_df.rename(columns={"DistrictCode": "DISTRICT_CODE", "Total": "EXPENSE"})

# Verify no null values
exp_df.isnull().sum()

# Verify no duplicate districts
exp_df[exp_df.duplicated(["DISTRICT_CODE"])]

# Add leading zeros to district code
exp_df["DISTRICT_CODE"] = exp_df["DISTRICT_CODE"].apply(lambda x: "{0:0>4}".format(x))

# Inspect expense values; clean and format expense column; replace missing values with None; cast as integer
exp_df.EXPENSE.unique()
exp_df = exp_df.replace("N", None)
exp_df["EXPENSE"] = exp_df["EXPENSE"].astype('int64')

# Verify values are of the correct type
exp_df.dtypes

DISTRICT_CODE    object
EXPENSE           int64
dtype: object

#### Clean test dataframe

In [72]:
# Drop and rename columns
test_df = test_df[["DistrictCode", "SchoolCode", "Test", "Subject", "School_Avg", "State_avg"]]
test_df = test_df.rename(columns={"DistrictCode": "DISTRICT_CODE", "SchoolCode": "SCHOOL_CODE",\
                                  "SchoolCode": "SCHOOL_CODE", "Test": "TEST","School_Avg": "SCH_AVG", \
                                  "State_avg": "STATE_AVG"})

# Verify no missing data
test_df.isnull().sum()

# Drop any duplicate rows
test_df.drop_duplicates()

# Add leading zeros to district and school codes
test_df["DISTRICT_CODE"] = test_df["DISTRICT_CODE"].apply(lambda x: "{0:0>4}".format(x))
test_df["SCHOOL_CODE"] = test_df["SCHOOL_CODE"].apply(lambda x: "{0:0>3}".format(x))


# Create unique key column from district and school codes
test_df["DS_CODE"] = test_df["DISTRICT_CODE"].map(str) + "-" + test_df["SCHOOL_CODE"].map(str)

# Review ACT scores to verify no missing values and scores within valid range
ACT_df = test_df[test_df['TEST'].str.contains('ACT')]
ACT_df.SCH_AVG.unique()
ACT_df.STATE_AVG.unique()

# Review SAT scores to verify no missing values and scores within valid range
SAT_df = test_df[test_df['TEST'].str.contains('SAT')]
SAT_df.SCH_AVG.unique()
SAT_df.STATE_AVG.unique()

# Replace missing values with None and cast as integer
test_df = test_df.replace(["N", "*"], None)
test_df["SCH_AVG"] = test_df["SCH_AVG"].astype('int64')

# Verify values are of the correct type
test_df.dtypes

# Filter only SAT scores and separate into columns for Math and English
test_df = test_df.loc[test_df['TEST'] == "SAT"]
math_df = test_df.loc[test_df["Subject"] == "Math"]
english_df = test_df.loc[test_df["Subject"] == "Reading and Writing"]
english_df = english_df[["DS_CODE", "Subject", "SCH_AVG", "STATE_AVG"]]
english_df = english_df.rename(columns={"SCH_AVG": "ENG_SCH_AVG", "STATE_AVG": "ENG_STATE_AVG"})
math_df = math_df.rename(columns={"SCH_AVG": "MATH_SCH_AVG", "STATE_AVG": "MATH_STATE_AVG"})
math_df = math_df[["Subject", "MATH_SCH_AVG", "MATH_STATE_AVG", "DS_CODE"]]
test_df = math_df.merge(english_df, on="DS_CODE", how="outer")
test_df = test_df.drop(labels = {"Subject_x", "Subject_y"}, axis = 1)

# Verify no missing data
test_df.isnull().sum()

DISTRICT_CODE    object
SCHOOL_CODE      object
TEST             object
Subject          object
SCH_AVG           int64
STATE_AVG         int64
DS_CODE          object
dtype: object

In [74]:
contact_df.to_csv(os.path.join('Data_temp', 'school_contact_cleaned.csv'))
ratio_df.to_csv(os.path.join('Data_temp', 'school_ratio_cleaned.csv'))
exp_df.to_csv(os.path.join('Data_temp', 'school_expense_cleaned.csv'))
test_df.to_csv(os.path.join('Data_temp', 'school_test_cleaned.csv'))